In [1]:
from numpy import pi

from qadence2_expressions import *
from qadence2_expressions.transform import collect_operators

## Basic symbol examples

In [2]:
a = Symbol('a')
b = Symbol('b')

exprs = [
    "a + a",
    "a - a",
    "a / a",
    "a + b",
    "a / (2*b)",
    "a ** 0",
    "a ** 1",
    "2 ** (a + b)",
]

for expr in exprs:
    print(expr, "->", eval(expr))

a + a -> 2 a
a - a -> 0
a / a -> 1
a + b -> a + b
a / (2*b) -> 0.5 a b^-1
a ** 0 -> 1
a ** 1 -> a
2 ** (a + b) -> 2^(a + b)


Product of sums is automatically expanded.

In [3]:
(a + b) * (a + b)

a^2 + 2 b a + b^2

However, power of sums are not automatically expanded to prioritise simplifications with respect to power.

In [4]:
(a + b) * (a + b) ** 2 / (a + b)

(a + b)^2

Minimal replace function. This functions still under development and may not present the expected behaviour.

In [5]:
x = a + b
y = a + 2 * cos(x) ** 2
z = replace(y, {a: 2})
x, y, z

(a + b, a + 2 cos(a + b)^2, 2 + 2 cos(2 + b)^2)

In [6]:
replace(z, {b: pi - 2})

4.0

Besides values, symbols can also be replaced by other symbols or expression.

In [7]:
replace(y, {a: 2*b, b: a})

2 b + 2 cos(2 b + a)^2

Entire terms of expression can also be replaced

In [8]:
replace(y, {b+a: 2*b})

a + 2 cos(2 b)^2

## Quantum operators

The `QSymbol` class allows to create arbitrary quantum operators as SU group element.

A `QSymbol` object is a callable object whose the arguments are the indices to what the operators are applied. For instance, `X(1)` is an `X` acting on qubit 1 while `H(1, 3)` indicates a Hadamard gate is applied to qubits 1 and 3. When no index is provide, like in `Z()` than it spans the gate to all qubits (regarding the number of qubits).

In [9]:
# Simple Hermitian operators
X = QSymbol('X')
Y = QSymbol('Y')
Z = QSymbol('Z')
SWAP = QSymbol('SWAP')

In [10]:
X(2) * Y() * Z(1,2)

X[2] Y[*] Z[1,2]

By default `QSymbols` are assumed Hermitian, non-hermitian operations can be set as so.

In [12]:
RX = lambda angle : QSymbol('Rx', angle, is_hermitian=False)

exprs = [
    "RX(0.5)() * RX(0.5)()",
    "RX(0.5)() * RX(0.5)().dag",
    "RX(0.5)() * RX(1.5)().dag",
    "RX(0.5)(1) * RX(0.5)(2).dag",
]

for expr in exprs:
    print(expr, "->", eval(expr))

RX(0.5)() * RX(0.5)() -> Rx(1.0)[*]
RX(0.5)() * RX(0.5)().dag -> 1
RX(0.5)() * RX(1.5)().dag -> Rx(-1.0)[*]
RX(0.5)(1) * RX(0.5)(2).dag -> Rx(0.5)[1] Rx'(0.5)[2]


**TO BE REPLACED**
By default the order of the indices doesn't matter, but it can set on to construct controlled gates like the following.

In [16]:
CNOT = QSymbol('CNOT', ordered_support=True)

CNOT(1, 2) * CNOT(1, 2), CNOT(1, 2) * CNOT(2, 1)

(1, CNOT[1,2] CNOT[2,1])

In [18]:
X * (cos(a) * X + 1j * sin(a) * Y) / 2

0.5 cos(a) + 0.5j sin(a) X[*] Y[*]

In [19]:
X(1) * (cos(a) * X() + 1j * sin(a) * Y()) * Z(1) / 2

0.5 cos(a) X[1] X[*] Z[1] + 0.5j sin(a) X[1] Y[*] Z[1]

In [20]:
Y(4) * X(3) * Y(5,4) * CNOT(1,2) * Z(3)

CNOT[1,2] X[3] Z[3] Y[4] Y[5,4]

In [22]:
nn = lambda k, l: (1 - Z(k)) * (1 - Z(l)) / 4
sum(a * X(i) - b * Z(i) + nn(i, i + 1) for i in range(2))

0.5 + a X[0] + -b Z[0] + -0.5 Z[1] + -0.25 Z[0] + 0.25 Z[0] Z[1] + a X[1] + -b Z[1] + -0.25 Z[2] + 0.25 Z[1] Z[2]

As an experimental feature, the `replace` functions can also used with `QSymbols`.

In [23]:
z = 2 * exp(X(1) + X(2))
replace(z, {exp(X(1) + X(2)): exp(X(1)) * exp(X(2))})

2 exp(X[1]) exp(X[2])

It's possible to capture the coeficient to the `QSymbols` in an expression using the `collect_operators` function.

In [24]:
h1 = Z(0) * Z(1)
h2 = Z(1) * Z(2)
h3 = cos(a) * h1 + 1j * sin(a) * h2
h4 = h1 - h3 + Z(1)

collect_operators(h4)

{Z[0] Z[1]: 1 + -cos(a), Z[1] Z[2]: -1j sin(a), Z[1]: 1}